In [1]:
#initial imports
import pandas as pd



In [2]:
#clean demographic data
# Replace 'your_excel_file.xlsx' with the path to your Excel file
excel_file = r'C:\Users\Owner\Desktop\research\TYP\summer2024TYP\muni\data\demog\cc-est2022-all(1).csv'

# Read the Excel file into a pandas DataFrame
df = pd.read_excel(excel_file)

# Display the columns
print("Columns in the Excel file:")
for column in df.columns:
    print(column)

ValueError: Excel file format cannot be determined, you must specify an engine manually.

In [ ]:
#merge and clean bond data

# Replace 'your_excel_file.xlsx' with the path to your Excel file

excel_file = r'C:\Users\Owner\Desktop\research\TYP\summer2024TYP\muni\data\bonds\concatenated_1823.xlsx'
# Read the Excel file into a pandas DataFrame
df = pd.read_excel(excel_file)

# Display the columns
print("Columns in the Excel file:")
for column in df.columns:
    print(column)